# **Analyzing Correlation Structures in a Model of Neural Activity-Dependent Homeostatic Plasticity (ADHP)**

# *Learning Objectives*

- Describe activity-dependent homeostatic plasticity (ADHP) and how it helps neurons maintain target activity levels and calcium concentrations
- Understand the concept of parameter space degeneracy in the context of neuron ionic channels (parameters) and membrane voltage (degenerate outcome)
- Implement a model of ADHP with various different sets of plasticity rules and explain how the rule sets differ
- Visually intuit why apparently anti-homeostatic rule sets still lead to homeostatic solutions
- Explain why choosing initial points from a relatively small area of parameter space can lead to spurious identification of correlation structure among the set of end points

# *Background*
## What are neural ionic currents?
Neuronal membranes, like all cell membranes, are generally impermeable to charged ions. However, neuronal membranes have special proteins called ion channels embedded within them which allow certain types of ions to pass through. When these ion channels are present, ions are allowed to move across the membrane according to two driving forces: concentration gradient and charge gradient. Like all chemical species, ions will tend to move away from areas where they have a high concentration and towards areas of low concentration due to entropy. Additionally, ions will be repelled by areas of the same charge and attracted to areas of the opposite charge. Based on the charge of an ion and its typical concentration in physiological situations, one can determine the membrane voltage at which the push and pull from the concentration and charge gradients will equalize. This is called the "reversal potential" of that ion current, and it is the point to which the ion current would drive the cell if it were acting on its own. 

Ionic currents with reversal potentials above the cell's typical resting potential are "depolarizing" or "excitatory", and currents with reversal potentials below the typical resting potential are "hyperpolarizing" or "inhibitory". For more a detailed overview of this topic, see Greg Conradi Smith's [Cellular Biophysics and Modeling](https://doi.org/10.1017/9780511793905). Depending on the number of ion channels of each type present (dictating the membrane's *conductance* of each current), the neuron's voltage will tend to equilibrate at some value. 

<img src="http://d1j63owfs0b5j3.cloudfront.net/term/images/1719-1529346321392.png" width="700">

[^Image courtesy of these General Biology flashcards^](https://www.drawittoknowit.com/course/general-biology/glossary/cellular-anatomy-physiology/electrochemical-gradient)

## What is activity-dependent homeostatic plasticity (ADHP)
It has been observed in various model systems that neurons which are chronically hypo- or hyper-active adjust their expression of various ion channels to restore target levels of excitation (Turrigiano et al., 1994). This process is thought to be dependent on intracellular Calcium concentration, which tracks activity level. As such, it is called activity-dependent homeostatic plasticity (ADHP). Putting aside the nonlinear properties of some ion channels which allow for complex neural behaviors like bursting, it makes logical sense that achieving such robustness would necessitate the following homeostatic rules:

Hyper-active neurons should downregulate their inward/excitatory conductances and upregulate their outward/inhibitory conductances. Hypo-active neurons should upregulate their inward/excitatory conductances and downregulate their outward/inhibitory ones. 

However, abstract models of ADHP proposed by O'Leary et al. (2013) have shown that other, more *counterintuitive* sets of rules can also confer robustness to certain perturbations. First, we will build some general intuitions about how ADHP works by trying to replicate this result using O'Leary et al.'s (2013) simple model. 

## Model Overview
In addition to its Calcium conductance, the model neuron contains three abstract ionic conductances: $g_1$, $g_2$,and $g_3$. Each one allows ions to flow across the membrane through a different type of protein channel. These channels don't have complex non-linear dynamics, meaning they don't exhibit gating or inactivation periods, like some realistic ion channels do. Each one is simply governed by one reversal potential, $E_1$, $E_2$, and $E_3$, respectively. The reversal potentials of these currents are defined as: $E_1=-90$, $E_2=-30$, and $E_3=+50$. Because the contribution of Calcium to the membrane potential is ignored in this model, the membrane potential ($Vm$) is given by:

>$C_m\frac{dVm}{dt} = g_1(-90-V_m) + g_2(-30-V_m) + g_3(50-V_m)$

where $C_m = 1$ is the membrane capacitance.

Based on the authors' earlier experimental work, the Calcium concentration ($[Ca^{2+}]$)was modeled as a function of the membrane potential. Specifically, 

>$\tau_{Ca}\frac{d[Ca^{2+}]}{dt} = A e^{b Vm} - [Ca^{2+}]$

where $\tau_{Ca} = 100$ is the time constant of the Calcium conductance, and $A = 109.2$, $b = 0.08$ are experimentally determined model parameters.

Finally, activity-dependent homeostatic plasticity (ADHP) occurs because the conductances themselves are changing as a function of the difference between the current Calcium concentration and the target Calcium concentration, $c_t$. For each of the three currents, $g_i$:

>$\tau_i\frac{dg_i}{dt} = g_i([Ca^{2+}] - c_t)$

where $\tau_i$ is the time constant of that conductance value. 

**_Importantly, these $\tau$'s determine in which direction and to what extent each conductance is affected by the distance from the target_**, which of course is the same for all conductances. A negative $\tau_i$ dictates that the conductance is increased when $[Ca^{2+}]$ is **too low** and a positive value dictates that the conductance is increased when $[Ca^{2+}]$ is **too high**. Additionally, a small $\tau_i$ indicates that that conductance is quickly and strongly responsive to Calcium changes, while a large tau dictates a slow and weak response. 

In summary, the system of equations governing this model system is:

>$C_m\frac{dVm}{dt} = g_1(-90-V_m) + g_2(-30-V_m) + g_3(50-V_m)$
>
>$\tau_{Ca}\frac{d[Ca^{2+}]}{dt} = A e^{b Vm} - [Ca^{2+}]$
>
>$\tau_1\frac{dg_1}{dt} = g_1([Ca^{2+}] - c_t)$
>
>$\tau_2\frac{dg_2}{dt} = g_2([Ca^{2+}] - c_t)$
>
>$\tau_3\frac{dg_3}{dt} = g_3([Ca^{2+}] - c_t)$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
go = plotly.graph_objs
import ipywidgets as widgets
from IPython.display import display
from sklearn.linear_model import LinearRegression


class ADHPModel:
    def __init__(self, initialValues = [-70, 0.2], parameterValues = [1,1,4000,-6000,-1000,100]):
        self.initialValues = initialValues
        self.t = 0
        # concentrations
        self.Vm = initialValues[0]
        self.Ca = initialValues[1]
        # parameters
        self.Ct = parameterValues[0]                                # (uM) target 
        self.Cm = parameterValues[1]                                # Membrane capacitance
        self.Tau1 = parameterValues[2]
        self.Tau2 = parameterValues[3]
        self.Tau3 = parameterValues[4]
        self.TauCa = parameterValues[5]  
        self.A = 109.2                            # uM (from experimental fitting)
        self.b = 0.08

    def simulate(self, dt, totalSimSteps, RandomStartPoints, plotTrajectory = 0, trajectoryStep = 100):
        self.convergence = np.zeros(len(RandomStartPoints[0]))
        self.numRandomStartPoints = len(RandomStartPoints[0])
        self.data = np.zeros((5,totalSimSteps))
        simTime = int(totalSimSteps*dt)
        self.time = np.linspace(0,simTime,totalSimSteps,endpoint=True)
        self.startingPoints = RandomStartPoints
        self.endingPoints = np.zeros((3,self.numRandomStartPoints))
        self.trajectory = np.zeros((3,self.numRandomStartPoints, int(totalSimSteps/trajectoryStep))) # setup trajectory plot. only need to measure every trajectoryStep iterations
        transientlen = int(200/dt)
        # for each point
        for i in range(self.numRandomStartPoints):
            self.Vm = -70
            self.Ca = .2
            [g1,g2,g3] = self.startingPoints[:,i]

            # first run out 200 seconds of transient where only [Ca] and Vm change
            for j in range(transientlen):
                self.Vm += dt*((g1*(-90-self.Vm) + g2*(-30-self.Vm) + g3*(50-self.Vm)) / self.Cm)
                self.Ca += dt*((self.A*(np.exp(self.b*self.Vm)) - self.Ca) / self.TauCa)

            # for each simulation step
            for j in range(totalSimSteps):
                self.Vm += dt*((g1*(-90-self.Vm) + g2*(-30-self.Vm) + g3*(50-self.Vm)) / self.Cm)
                self.Ca += dt*((self.A*(np.exp(self.b*self.Vm)) - self.Ca) / self.TauCa)
                g1 += dt*(g1*(self.Ca - self.Ct) / self.Tau1)
                g2 += dt*(g2*(self.Ca - self.Ct) / self.Tau2)
                g3 += dt*(g3*(self.Ca - self.Ct) / self.Tau3)
                self.data[:,j] = np.array([self.Ca,self.Vm,g1,g2,g3])
                self.t += dt
                if (plotTrajectory == 1) and (j % trajectoryStep == 0): self.trajectory[:,i,int(j/trajectoryStep)] = np.array([g1,g2,g3])
            self.endingPoints[:,i] = np.array([g1,g2,g3])
            if self.Ca < 1.1 and self.Ca > .9:
                self.convergence[i] = 1
        self.data = np.vstack((np.arange(0,totalSimSteps), self.data))

Tauoptions = widgets.ToggleButtons(
    options=['Original', 'Flipped', 'Scaled','Custom'],
    description='Tau set:',
    disabled=False,
    button_style='',
    tooltips=['Tau1 = 4000,Tau2 = -6000,Tau3 = -1000', 'Tau1 = 4000,Tau2 = 6000,Tau3 = -1000', 'Tau1 = 4000,Tau2 = -6000/10,Tau3 = -1000/40'],
)
Vmbox = widgets.Checkbox(
    value=False,
    description='Plot Mem. Voltage',
    disabled=False,
    indent=False,
    continuous_update=True
)
g1box = widgets.Checkbox(
    value=True,
    description='Plot g1',
    disabled=False,
    indent=False,
    continuous_update=True
)
g2box = widgets.Checkbox(
    value=True,
    description='Plot g2',
    disabled=False,
    indent=False,
    continuous_update=True
)
g3box = widgets.Checkbox(
    value=True,
    description='Plot g3',
    disabled=False,
    indent=False,
    continuous_update=True
)
Cabox = widgets.Checkbox(
    value=False,
    description='Plot [Ca]',
    disabled=False,
    indent=False,
    continuous_update=True
)
Tau1in = widgets.BoundedFloatText(
    value=4000,
    min=-9000,
    max=9000,
    step=100,
    description='Custom Tau1:',
    disabled=False,
    continuous_update=False
)
Tau2in = widgets.BoundedFloatText(
    value=-6000,
    min=-9000,
    max=9000,
    step=100,
    description='Custom Tau2:',
    disabled=False,
    continuous_update=False
)
Tau3in = widgets.BoundedFloatText(
    value=-1000,
    min=-9000,
    max=9000,
    step=100,
    description='Custom Tau3:',
    disabled=False,
    continuous_update=False
)
simtimein = widgets.BoundedIntText(
    value=2000,
    min = 400,
    max = 100000,
    step = 50,
    description = 'Sim. Time (s)',
    disabled = False,
    continuous_update = True
)

# Module A: Effect of Homoestatic Rules on the Trajectory of a Single Neuron
This neruon, which has the conductance values of g1 = 100, g2 = 20, g3 = 10, is too hyperpolarized. It's membrane voltage and calcium concentration lie too far below their target values. 

## **1) Interact with the buttons below to answer the following questions**

*The simulations take a moment to compute, so after clicking on a button, wait 2-4 seconds to see the changed result.*

- Select the "Original" button to set $\tau_1$ = 4000, $\tau_2$ = -6000, and $\tau_3$ = -1000. Given the reversal potentials of each of the three currents relative to the neuron's equilibrium voltage, do these three tau's "make sense"? In other words, do they indicate the currents should change in the direction you would expect? Now, observe the simulation of the neuron as it undergoes homeostatic plasticity for 2000 seconds. How do the conductances change? At equilibrium, is the target calcium concentration of 1 uM achieved? 

- Select the "Flipped" button to flip the sign of $\tau_2$. Does this set of tau's still "make sense" with respect to the reversal potentials of the currents? Do they still guide the neuron towards its target calcium conentration of 1 uM?

- Select the "Scaled" button to scale $\tau_2$ and $\tau_3$ ($\tau_2$ = -6000/10 = -600, and $\tau_3$ = -1000/40 = -25). Notice that once ADHP turns on following a short transient, there is some oscillatory activity before the neuron settles down again. Why might this be? Consider the conductance value that is driving this wobble. How fast is it now changing relative to the calcium concentration and membrane capacitance (Capacitance = 1, $\tau_{Ca}$ = 100)?

- Select the "Custom" button and manually adjust the values in the boxes for $\tau_1$, $\tau_2$, and $\tau_3$. Can you find a set of tau's which do *not* successfully rescue the neuron from this initial condition? O'Leary et al. (2013) found that about 62% of the random tau sets they tried from this initial condition did help the neuron converge to its target calcium concentration. Does this seem consistent with your tests?

In [ ]:
@widgets.interact(tauset=Tauoptions,Vmbox=Vmbox,g1box=g1box,g2box=g2box,g3box=g3box,Cabox=Cabox,tau1in=Tau1in,tau2in=Tau2in,tau3in=Tau3in,simtime=simtimein)
def singleIC(tauset,Vmbox,g1box,g2box,g3box,Cabox,tau1in=4000,tau2in=-6000,tau3in=-1000,simtime=1000):
    elementstoplot=[Cabox,Vmbox,g1box,g2box,g3box]
    Neuron = ADHPModel()
    trajcolor='blue'
    if tauset == 'Scaled':
        trajcolor = 'green'
        Neuron.Tau2 = Neuron.Tau2/10
        Neuron.Tau3 = Neuron.Tau3/40
    if tauset == 'Flipped':
        trajcolor = 'red'
        Neuron.Tau2 = -Neuron.Tau2
    if tauset == 'Custom':
        trajcolor = 'black'
        Neuron.Tau1=Tau1in.value
        Neuron.Tau2=Tau2in.value
        Neuron.Tau3=Tau3in.value
    dt = .005
    simsteps = int(simtime/dt)
    Neuron.simulate(dt, simsteps, np.array([[100],[20],[10]]) , plotTrajectory = 1, trajectoryStep = 10)
    plt.rc('font',size=14)
    fig = plt.figure(figsize=(30,10))
    ax1 = fig.add_subplot(1,2,1)
    labels=['[Ca]','Vm','g1','g2','g3']
    colors = ['black','gray','c','m','y']
    for i in range(len(elementstoplot)):
        if elementstoplot[i]:
            ax1.plot(Neuron.time,Neuron.data[i+1,:],label=labels[i],color=colors[i])
    ax1.legend()
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Value (mV,μS/nF,μM)')
    ax2 = fig.add_subplot(1,2,2,projection='3d')
    ax2.view_init(elev=40, azim=115)
    ax2.plot(Neuron.trajectory[0,0,:], Neuron.trajectory[1,0,:], Neuron.trajectory[2,0,:], color=trajcolor, alpha=0.5)
    ax2.tick_params(axis='both', which='major', labelsize=10)
    ax2.set_xlabel('g1')
    ax2.set_ylabel('g2')
    ax2.set_zlabel('g3')
    plt.show()
    print("Tau1=",Neuron.Tau1," Tau2=",Neuron.Tau2," Tau3=",Neuron.Tau3,"*")
    print('*Custom taus not used unless "Custom" is highlighted above')
    print()
    print('Final [Ca] (μM):',Neuron.Ca)
    print('Final Voltage (mV):',Neuron.Vm)

_**Debrief:**_ There are indeed many different sets of values for [$\tau_1$,$\tau_2$,$\tau_3$] which could guide this neuron to a stable equilibrium at its target Calcium concentration, even though some don't "make sense". All that is necessary is that the homoestatic rule pushes the neuron in a direction in conductance space (3D trajectory on the right) towards an area where this condition will be satisfied. 

In this case, that area is an infinite hyper-plane of equilibrium points (see the pink plane from O'Leary et al.'s Figure 1), each of which yields the target Calcium concentration, but with a different combination of conductances ($g_1, g_2, g_3$). This phenomenon is known as *degeneracy of solutions* because many different sets of parameters yield similar outcomes. 

<img src="https://nanohub.org/resources/adhp/download/eqplane.png" alt="O'Leary et al., 2013 Figure 1"/>

Real neurons, which likely exhibit functional ADHP mechanisms, are thus found to maintain themselves around some consistent Calcium concentration. This Calcium concentration can be achieved in certain areas of conductance space which constitute a degenerate set of solutions. Moreover, upon measuring the different conductance values of these neurons, recognizable _"correlation structures"_ are observed (i.e. some conductance values are reliably positively correlated with certain others, negatively correlated with certain others, etc.). Is this just a result of the degeneracy of parameter space, or is the degeneracy further constrained by the rules of the ADHP mechanism that maintains the neuron? 

To investigate this question with this model, we should:  

(1) start many neurons off with initial conductance values that do not produce the target voltage  

(2) allow different ADHP mechanisms to act on them  

(3) observe their equilibrium conductance values   

Will they simply be distributed across the plane of good solutions, or will there exist some more definable structure that is attributable to the rule sets? 


# Module B: Simulate the Effect of Several Different ADHP Mechanisms on a Sample of Neurons


## **2) Interact with the buttons below to do the following**

- Click on "Simulate" to generate three samples of 10 neurons, with conductance values centered around the point $(g_1, g_2, g_3) = (105,20,10)$, and a variance of $2.5$ units in each dimension. Each of the three ADHP rules you have explored ("Original", "Flipped", and "Scaled") will be applied to one sample. This is the exact procedure completed by O'Leary et al. (2013). 

If you get an overflow error with your custom tau set, try reducing the step size of the simulation, though this decreases the speed. If not all the points reach the target [Ca], but you expect them to, try increasing the simulation duration.


In [ ]:
num_pts_slide = widgets.IntSlider(
    value=10,
    min=0,
    max=50,
    step=5,
    description='Num. of Initial Pts:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
variance_slide = widgets.FloatSlider(
    value=2.5,
    min=.5,
    max=10,
    step=.1,
    description='Sample Variance:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
g1centerin = widgets.BoundedFloatText(
    value=105,
    min=0,
    max=200,
    step=1,
    description='g1 centered on:',
    disabled=False
)
g2centerin = widgets.BoundedFloatText(
    value=20,
    min=0,
    max=200,
    step=1,
    description='g2 centered on:',
    disabled=False
)
g3centerin = widgets.BoundedFloatText(
    value=10,
    min=0,
    max=200,
    step=1,
    description='g3 centered on:',
    disabled=False
)
OGcheckbox =widgets.Checkbox(
    value=True,
    description='Original Taus',
    disabled=False,
    indent=False
)
Flippedcheckbox = widgets.Checkbox(
    value=True,
    description='Flipped Taus',
    disabled=False,
    indent=False
)
Scaledcheckbox = widgets.Checkbox(
    value=True,
    description='Scaled Taus',
    disabled=False,
    indent=False
)
Customcheckbox = widgets.Checkbox(
    value=False,
    description='Custom Taus',
    disabled=False,
    indent=False
)

Stepsizeslide = widgets.BoundedFloatText(
    value=.01,
    min=0.0001,
    max=1,
    step=0.0001,
    description='Step Size',
    disabled=False
)

Simulationdurationslide = widgets.IntSlider(
    value=3000,
    min=1000,
    max=10000,
    step=1000,
    description='Sim. Dur.',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def simulatemodels(OG,Flipped,Scaled,Custom,tau1in=4000,tau2in=-6000,tau3in=-1000,num_pts=10,variance=2.5,centerg1=105,centerg2=20,centerg3=10,stepsize=0.01,simtime=3000):
    '''run simulations indicated'''
    randomICs = np.vstack((np.random.normal(centerg1,variance,size=num_pts),np.random.normal(centerg2,variance,size=num_pts),np.random.normal(centerg3,variance,size=num_pts)))
    models = []
    simsteps = int(simtime/stepsize)
    if OG:
        global originalModel
        originalModel = ADHPModel()
        originalModel.simulate(stepsize, simsteps, randomICs, 1, 100)
        print("OG simluated")
        if (originalModel.convergence.all()):
            print("All points reached target [Ca]")
        else:
            print("Note: not all points reached target [Ca]")
    if Flipped:
        global flippedModel
        flippedModel = ADHPModel(parameterValues=[1,1,4000,6000,-1000,100])
        flippedModel.simulate(stepsize, simsteps, randomICs, 1, 100)
        print("Flipped simulated")
        if (flippedModel.convergence.all()):
            print("All points reached target [Ca]")
        else:
            print("Note: not all points reached target [Ca]")
    if Scaled:
        global scaledModel
        scaledModel = ADHPModel(parameterValues=[1,1,4000,-6000/10,-1000/40,100])
        scaledModel.simulate(stepsize, simsteps, randomICs, 1, 100)
        print("Scaled simulated")
        if (scaledModel.convergence.all()):
            print("All points reached target [Ca]")
        else:
            print("Note: not all points reached target [Ca]")
    if Custom:
        global customModel
        customModel = ADHPModel(parameterValues=[1,1,tau1in,tau2in,tau3in,100])
        customModel.simulate(stepsize, simsteps, randomICs, 1, 100)
        print("Custom simulated")
        if (customModel.convergence.all()):
            print("All points reached target [Ca]")
        else:
            print("Note: not all points reached target [Ca]")

def plotsamplepts():
    '''Plot start/end points'''
    endpts = np.empty(0)
    num_pts = num_pts_slide.value
    colors = np.tile('orange',num_pts)
    if OGcheckbox.value:
        startpts = originalModel.startingPoints
        endpts = originalModel.endingPoints
        colors = np.concatenate((colors,np.tile('blue',num_pts)))
    if Flippedcheckbox.value:
        startpts = flippedModel.startingPoints
        if len(endpts) == 0:
            endpts = flippedModel.endingPoints
        else:
            endpts = np.concatenate((endpts,flippedModel.endingPoints),axis=1)
        colors = np.concatenate((colors,np.tile('red',num_pts)))
    if Scaledcheckbox.value:
        startpts = scaledModel.startingPoints
        if len(endpts) == 0:
            endpts = scaledModel.endingPoints
        else:
            endpts = np.concatenate((endpts,scaledModel.endingPoints),axis=1)
        colors = np.concatenate((colors,np.tile('green',num_pts)))
    if Customcheckbox.value:
        startpts = customModel.startingPoints
        if len(endpts) == 0:
            endpts = customModel.endingPoints
        else:
            endpts = np.concatenate((endpts,customModel.endingPoints),axis=1)
        colors = np.concatenate((colors,np.tile('black',num_pts)))
    pts = np.concatenate((startpts,endpts),axis=1)
      
    fig= go.Figure(data=[go.Scatter3d(x=pts[0], y=pts[1], z=pts[2],mode='markers',marker=dict(
        size=5,
        color=colors,                # set color to an array/list of desired values
        opacity=0.8))
        ])
    fig.update_layout(scene = dict(
                    xaxis_title='g1',
                    yaxis_title='g2',
                    zaxis_title='g3'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
    #fig.update_zaxes(title_text="g3")
    fig.show()

In [ ]:
sim_button = widgets.Button(description="Simulate")
output = widgets.Output()


@output.capture(clear_output=True)
def on_sim_button_clicked(b):
    with output:
        print("Simulating...")
        simulatemodels(OG=OGcheckbox.value,Flipped=Flippedcheckbox.value,Scaled=Scaledcheckbox.value,Custom=Customcheckbox.value,tau1in=Tau1in.value,tau2in=Tau2in.value,tau3in=Tau3in.value,num_pts = num_pts_slide.value,variance=variance_slide.value,centerg1=g1centerin.value,centerg2=g2centerin.value,centerg3=g3centerin.value,stepsize=Stepsizeslide.value,simtime=Simulationdurationslide.value)
        print("Simulation Complete, you may now plot")

display(OGcheckbox,Flippedcheckbox,Scaledcheckbox,Customcheckbox,Tau1in,Tau2in,Tau3in,num_pts_slide,variance_slide,g1centerin,g2centerin,g3centerin,Stepsizeslide,Simulationdurationslide,sim_button,output)
sim_button.on_click(on_sim_button_clicked)

## __*Note: Do not continue on until the simulation is complete.__

## **3) Click "Plot Pts" to visualize your initial samples and where each ended up after ADHP**

- By dragging the plot around, can you identify the plane of degenerate solutions referenced earlier? Does it appear that all the initial points converged there?

- Does it look to you like there might be additional correlational structure?


In [ ]:
plot_button = widgets.Button(description="Plot Pts")

output1 = widgets.Output()
@output1.capture(clear_output=True)
def on_plot_button_clicked(b):
    with output1:
        print("Plotting...")
        print("Legend: \n","Initial Pts = Orange\n","Original = Blue\n", "Flipped = Red\n", "Scaled = Green\n", "Custom = Black")      
        plotsamplepts()

display(plot_button)
display(output1)
plot_button.on_click(on_plot_button_clicked)

## **4) Click "Plot Trajectories" to plot each point's trajectory through parameter space**

- It's a bit more clunky, but you can adjust the angle of plot viewing with the Elevation and Azimuth sliders and then re-clicking "Plot Trajectories". Can you still identify the same plane of steady states? Is it clear from the directions of movement why the different solution clusters ended up where they did within the plane? 

- Can you identify from the trajectories which tau set induced the oscillatory "wobble" behavior?

In [ ]:
elevslide = widgets.FloatSlider(
    value=40,
    min=0,
    max=360,
    step=10,
    description='Elevation:',
    disabled=False,
    continuous_update=False,
)
azimslide = widgets.FloatSlider(
    value=100,
    min=0,
    max=360,
    step=10,
    description='Azimuth:',
    disabled=False,
    continuous_update=False,
)

In [ ]:
def plottrajectories(elev,azim):
    fig = plt.figure(figsize = (30,15))
    ax = plt.axes(projection="3d")
    ax.view_init(elev=elev, azim=azim)

    models = []
    colors = []
    if OGcheckbox.value:
        models.append(originalModel)
        colors.append('blue')
    if Flippedcheckbox.value:
        models.append(flippedModel)
        colors.append('red')
    if Scaledcheckbox.value:
        models.append(scaledModel)
        colors.append('green')
    if Customcheckbox.value:
        models.append(customModel)
        colors.append('black')

    for i in range(len(models)):
        Model = models[i]
        color = colors[i]
        for p in range(num_pts_slide.value):
            ax.plot3D(Model.trajectory[0,p,:]/1000, Model.trajectory[1,p,:]/1000, Model.trajectory[2,p,:]/1000, color=color, alpha=0.5)
        ax.scatter3D(Model.startingPoints[0,:]/1000,Model.startingPoints[1,:]/1000,Model.startingPoints[2,:]/1000,color='orange',s=40)
        ax.scatter3D(Model.endingPoints[0,:]/1000,Model.endingPoints[1,:]/1000,Model.endingPoints[2,:]/1000,color=color,s=40)
    ax.set_xlabel('g1')
    ax.set_ylabel('g2')
    ax.set_zlabel('g3')
    plt.show()

traj_button = widgets.Button(description="Plot Trajectories")

outputtraj = widgets.Output()
@outputtraj.capture(clear_output=True)
def on_traj_button_clicked(b):
    with outputtraj:
        print("Plotting...")
        print("Legend: \n","Initial Pts = Orange\n","Original = Blue\n", "Flipped = Red\n", "Scaled = Green\n", "Custom = Black")      
        plottrajectories(elevslide.value,azimslide.value)

display(traj_button,elevslide,azimslide)
display(outputtraj)
traj_button.on_click(on_traj_button_clicked)

# Module C: Analyze and contrast correlation structure for small and large spreads of initial conditions

## **5) Interact with the buttons below to toggle between the pairwise correlation graphs for the data sets you've simulated**

- Are there strong correlations between the conductance values of the solutions (high r-values)? 

- Does the situation look similar or different between the "Original", "Flipped", and "Scaled" conditions? How many of the pairwise correlations have the same direction?

In [ ]:
corrplot = widgets.RadioButtons(
    options=['Original','Flipped','Scaled','Custom'],
    value='Original',
    description='Display Correlation:',
    disabled=False,
    continuous_update=False
)

def pairwisecorr(modelstr):
    plotting=True
    if modelstr == 'Original':
        if OGcheckbox.value:
            model = originalModel
            color = 'blue'
        else: 
            raise ValueError("You cannot plot correlations for ADHP model data you have not simulated")
    if modelstr == 'Flipped':
        if Flippedcheckbox.value:
            model = flippedModel
            color = 'red'
        else: 
            raise ValueError("You cannot plot correlations for ADHP model data you have not simulated")
    if modelstr == 'Scaled':
        if Scaledcheckbox.value:
            model = scaledModel
            color = 'green'
        else: 
            raise ValueError("You cannot plot correlations for ADHP model data you have not simulated")
    if modelstr == 'Custom':
        if Customcheckbox.value:
            model = customModel
            color = 'black'
        else: 
            raise ValueError("You cannot plot correlations for ADHP model data you have not simulated")
    corrmatrix = np.ones((3,3))
    coefficients = np.zeros((3,3))
    if plotting:
        fig = plt.figure(figsize=(13, 5))
        plt.rc('font',size=14)
        ax1 = plt.subplot(331)
        ax2 = plt.subplot(332)
        ax3 = plt.subplot(333)
        ax4 = plt.subplot(334)
        ax5 = plt.subplot(335)
        ax6 = plt.subplot(336)
        ax7 = plt.subplot(337)
        ax8 = plt.subplot(338)
        ax9 = plt.subplot(339)
        axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9]
        axes=np.reshape(axes,(3,3))
        labels = ['g1','g2','g3']
    for i in range(3):
        for j in range(3):
            axes[i,j].tick_params(axis='both', which='major', labelsize=8)
            if j == 0:
                axes[i,j].set_ylabel(labels[i])
            if i == 2:
                axes[i,j].set_xlabel(labels[j])
            if i != j:
                a = model.endingPoints[i]
                A=np.reshape(a,(-1,1))
                b=model.endingPoints[j]
                reg = LinearRegression().fit(A,b)
                corrmatrix[i,j] = reg.score(A,b)
                coefficients[i,j] = reg.coef_
                if plotting:
                    axes[i,j].scatter(a,b,color=color)
                    axes[i,j].plot(a,reg.predict(A),color=color)
    if plotting:
        plt.show()
        fig1 = plt.figure(figsize=(15,5))
        ax = plt.subplot(111)
        axp = plt.imshow(corrmatrix,origin = 'upper',  extent = [0, 25, 0, 10])
        plt.title('Correlation strengths (R^2)')
        plt.xticks(ticks=[])
        plt.yticks(ticks=[])
        cb = plt.colorbar(axp,ax=[ax],location='left',shrink=.85)
        plt.show()
    return

corr_button = widgets.Button(description="Show Correlations")

outputcorr = widgets.Output()
@outputcorr.capture(clear_output=True)
def on_corr_button_clicked(b):
    with outputcorr:
        pairwisecorr(corrplot.value)

display(corrplot,corr_button)
display(outputcorr)
corr_button.on_click(on_corr_button_clicked)

*Debrief:* Relatively strong correlation structure has emerged, though it doesn't differ much between conditions. It makes sense that the set of $\tau_i$'s you pick should define a general direction of movement in conductance space, but why should that result in a correlation structure among the resulting solutions when the whole plane of equilibrium points is available? It seems like this may be a result of choosing initial conditions from a very small area of parameter space far away from the plane of equilibrium points, and when acted on by a direction of movement, these points spread out in a way that gave them correlational structure. Thus, it is **not anything specific about the ADHP rules that lead to this, but only the initial conditions.** We can test this hypothesis by asking if the same patterns emerge for a more spread-out cluster of initial points which is not so far away from the equilibrium plane

## **6)Go back and repeat steps 2-5, but this time with a different distribution of initial points.**

- Change the center of the point cluster by typing in values for $g_1, g_2$, and $g_3$ which are close to or on the plane of equilibrium solutions (for example, centered around (90,20,20)). Then, increase the sample variance to something like 15 and hit "Simulate". Follow the same steps and answer the same questions in parts 3 and 4. 

*Debrief:* Now, the points appear to be more spread out on the plane, with less definable structure.


## **Conclusion**: 
Through these modules, we have seen that multiple sets of rules governing activity-dependent homeostatic plasticity can lead hyper- or hypo-active neurons back to their target activity level. This includes rule sets that we wouldn't expect to be homeostatic, but because they still lead the neuron in a favorable direction in parameter space, they restore homeostasis. From a sample of initial points with a small variance, each ADHP mechanism yielded a distinct cluster of final solutions on the equilibrium plane. It has been hypothesized in the literature that these distinct correlation structures are the result of the unique homeostatic rules. However, when the variance of this sample was increased, the correlation structure disappears, with the end results of each ADHP mechanism being virtually indistinguishable. Thus, it is only the degeneracy of the parameter space, not the ADHP mechanism, which determines correlation structure of the final solutions. 

## **Future questions**:
*   We have seen cases where a steady state is reached, but not at the correct calcium concentration. Are there initial conditions (especially in the flipped/anti-homeostatic case) where a steady state is not reached (or all conductances go to zero)? 

* Would noise in the system disrupt these results? 

* What is the basin of attraction of the $Ca^{2+}$ steady state plane, and how does it differ under different homeostatic rules?

* What are some other measures that could be used to numerically demonstrate the difference between the small variance and large variance case? (Ideas include clustering algorithms, linear separability/machine learning, information theory, etc.)


# *References*

O’Leary, T., Williams, A. H., Caplan, J. S., & Marder, E. (2013). Correlations in ion channel expression emerge from homeostatic tuning rules. Proceedings of the National Academy of Sciences, 110(28). https://doi.org/10.1073/pnas.1309966110

Turrigiano, G., Abbott, L. F., & Marder, E. (1994). Activity-dependent changes in the intrinsic properties of cultured neurons. Science, 264(5161), 974–977. https://doi.org/10/fpjxb5

# *Additional Reading*

Caplan, J. S., Williams, A. H., & Marder, E. (2014). Many parameter sets in a multicompartment model oscillator are robust to temperature perturbations. The Journal of Neuroscience: The Official Journal of the Society for Neuroscience, 34(14), 4963–4975. https://doi.org/10.1523/JNEUROSCI.0280-14.2014

Conradi Smith, G. (2019). Cellular Biophysics and Modeling: A Primer on the Computational Biology of Excitable Cells. Cambridge: Cambridge University Press. https://doi.org/10.1017/9780511793905

O’Leary, T., Williams, A. H., Franci, A., & Marder, E. (2014). Cell Types, Network Homeostasis, and Pathological Compensation from a Biologically Plausible Ion Channel Expression Model. Neuron, 82(4), 809–821. https://doi.org/10.1016/j.neuron.2014.04.002

Santin, J. M., & Schulz, D. J. (2019). Membrane Voltage Is a Direct Feedback Signal That Influences Correlated Ion Channel Expression in Neurons. Current Biology, 29(10), 1683-1688.e2. Scopus. https://doi.org/10.1016/j.cub.2019.04.008

Schulz, D. J., Goaillard, J.-M., & Marder, E. (2006). Variable channel expression in identified single and electrically coupled neurons in different animals. Nature Neuroscience, 9(3), Article 3. https://doi.org/10.1038/nn1639

Temporal, S., Desai, M., Khorkova, O., Varghese, G., Dai, A., Schulz, D. J., & Golowasch, J. (2012). Neuromodulation independently determines correlated channel expression and conductance levels in motor neurons of the stomatogastric ganglion. Journal of Neurophysiology, 107(2), 718–727. https://doi.org/10.1152/jn.00622.2011

Temporal, S., Lett, K. M., & Schulz, D. J. (2014). Activity-dependent feedback regulates correlated ion channel mRNA levels in single identified motor neurons. Current Biology: CB, 24(16), 1899–1904. https://doi.org/10.1016/j.cub.2014.06.067